In [18]:
import pandas as pd
import os

pwd = os.getcwd()
filepath = os.path.join(pwd, "Placement_Data_Full_Class.csv")

data_import = pd.read_csv(filepath)
data_import

,sl_no,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,salary
0,1,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed,270000.0
1,2,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed,200000.0
2,3,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed,250000.0
3,4,M,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed,NaN
4,5,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed,425000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,211,M,80.60,Others,82.00,Others,Commerce,77.60,Comm&Mgmt,No,91.0,Mkt&Fin,74.49,Placed,400000.0
211,212,M,58.00,Others,60.00,Others,Science,72.00,Sci&Tech,No,74.0,Mkt&Fin,53.62,Placed,275000.0
212,213,M,67.00,Others,67.00,Others,Commerce,73.00,Comm&Mgmt,Yes,59.0,Mkt&Fin,69.72,Placed,295000.0
213,214,F,74.00,Others,66.00,Others,Commerce,58.00,Comm&Mgmt,No,70.0,Mkt&HR,60.23,Placed,204000.0


In [19]:
data_import.describe()

,sl_no,ssc_p,hsc_p,degree_p,etest_p,mba_p,salary
count,215.000000,215.000000,215.000000,215.000000,215.000000,215.000000,148.000000
mean,108.000000,67.303395,66.333163,66.370186,72.100558,62.278186,288655.405405
std,62.209324,10.827205,10.897509,7.358743,13.275956,5.833385,93457.452420
min,1.000000,40.890000,37.000000,50.000000,50.000000,51.210000,200000.000000
25%,54.500000,60.600000,60.900000,61.000000,60.000000,57.945000,240000.000000
50%,108.000000,67.000000,65.000000,66.000000,71.000000,62.000000,265000.000000
75%,161.500000,75.700000,73.000000,72.000000,83.500000,66.255000,300000.000000
max,215.000000,89.400000,97.700000,91.000000,98.000000,77.890000,940000.000000


In [20]:
dropped_cols = data_import.copy()

dropped_cols = dropped_cols.drop(["sl_no", "salary"], axis=1)
dropped_cols

,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status
0,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed
1,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed
2,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed
3,M,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed
4,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed
...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,M,80.60,Others,82.00,Others,Commerce,77.60,Comm&Mgmt,No,91.0,Mkt&Fin,74.49,Placed
211,M,58.00,Others,60.00,Others,Science,72.00,Sci&Tech,No,74.0,Mkt&Fin,53.62,Placed
212,M,67.00,Others,67.00,Others,Commerce,73.00,Comm&Mgmt,Yes,59.0,Mkt&Fin,69.72,Placed
213,F,74.00,Others,66.00,Others,Commerce,58.00,Comm&Mgmt,No,70.0,Mkt&HR,60.23,Placed


In [21]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

labeled_data = dropped_cols.copy()

cols_to_labelled = ["gender", "workex", "ssc_b", "hsc_b", "specialisation"]

labeled_data[cols_to_labelled] = labeled_data[cols_to_labelled].apply(le.fit_transform)

labeled_data

,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status
0,1,67.00,1,91.00,1,Commerce,58.00,Sci&Tech,0,55.0,1,58.80,Placed
1,1,79.33,0,78.33,1,Science,77.48,Sci&Tech,1,86.5,0,66.28,Placed
2,1,65.00,0,68.00,0,Arts,64.00,Comm&Mgmt,0,75.0,0,57.80,Placed
3,1,56.00,0,52.00,0,Science,52.00,Sci&Tech,0,66.0,1,59.43,Not Placed
4,1,85.80,0,73.60,0,Commerce,73.30,Comm&Mgmt,0,96.8,0,55.50,Placed
...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,1,80.60,1,82.00,1,Commerce,77.60,Comm&Mgmt,0,91.0,0,74.49,Placed
211,1,58.00,1,60.00,1,Science,72.00,Sci&Tech,0,74.0,0,53.62,Placed
212,1,67.00,1,67.00,1,Commerce,73.00,Comm&Mgmt,1,59.0,0,69.72,Placed
213,0,74.00,1,66.00,1,Commerce,58.00,Comm&Mgmt,0,70.0,1,60.23,Placed


In [22]:
hot_encoded_data = labeled_data.copy()

hot_encoded_data_temp = hot_encoded_data["status"]
hot_encoded_data = hot_encoded_data.drop("status", axis=1)

hot_encoded_data = pd.get_dummies(hot_encoded_data)

hot_encoded_data = pd.concat([hot_encoded_data, hot_encoded_data_temp], axis=1)
hot_encoded_data

,gender,ssc_p,ssc_b,hsc_p,hsc_b,degree_p,workex,etest_p,specialisation,mba_p,hsc_s_Arts,hsc_s_Commerce,hsc_s_Science,degree_t_Comm&Mgmt,degree_t_Others,degree_t_Sci&Tech,status
0,1,67.00,1,91.00,1,58.00,0,55.0,1,58.80,False,True,False,False,False,True,Placed
1,1,79.33,0,78.33,1,77.48,1,86.5,0,66.28,False,False,True,False,False,True,Placed
2,1,65.00,0,68.00,0,64.00,0,75.0,0,57.80,True,False,False,True,False,False,Placed
3,1,56.00,0,52.00,0,52.00,0,66.0,1,59.43,False,False,True,False,False,True,Not Placed
4,1,85.80,0,73.60,0,73.30,0,96.8,0,55.50,False,True,False,True,False,False,Placed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,1,80.60,1,82.00,1,77.60,0,91.0,0,74.49,False,True,False,True,False,False,Placed
211,1,58.00,1,60.00,1,72.00,0,74.0,0,53.62,False,False,True,False,False,True,Placed
212,1,67.00,1,67.00,1,73.00,1,59.0,0,69.72,False,True,False,True,False,False,Placed
213,0,74.00,1,66.00,1,58.00,0,70.0,1,60.23,False,True,False,True,False,False,Placed


In [23]:
#scale data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_data = hot_encoded_data.copy()
labels = scaled_data.iloc[:, -1] #omit last col since those are our labels
scaled_data = scaled_data.iloc[:, :-1]

scaled_data = scaler.fit_transform(scaled_data)
scaled_data

array([[ 0.73943397, -0.02808697,  1.08245885, ..., -1.43924583,
        -0.23221018,  1.62605898],
       [ 0.73943397,  1.11336869, -0.92382264, ..., -1.43924583,
        -0.23221018,  1.62605898],
       [ 0.73943397, -0.21323793, -0.92382264, ...,  0.69480833,
        -0.23221018, -0.61498384],
       ...,
       [ 0.73943397, -0.02808697,  1.08245885, ...,  0.69480833,
        -0.23221018, -0.61498384],
       [-1.35238581,  0.61994138,  1.08245885, ...,  0.69480833,
        -0.23221018, -0.61498384],
       [ 0.73943397, -0.49096436, -0.92382264, ...,  0.69480833,
        -0.23221018, -0.61498384]])

In [24]:
from sklearn.model_selection import train_test_split

X = scaled_data
y = labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train

array([[-1.35238581,  1.08281878,  1.08245885, ..., -1.43924583,
        -0.23221018,  1.62605898],
       [ 0.73943397,  0.86989518, -0.92382264, ...,  0.69480833,
        -0.23221018, -0.61498384],
       [ 0.73943397,  1.36054521,  1.08245885, ..., -1.43924583,
        -0.23221018,  1.62605898],
       ...,
       [ 0.73943397, -0.49096436,  1.08245885, ...,  0.69480833,
        -0.23221018, -0.61498384],
       [ 0.73943397,  0.61994138,  1.08245885, ...,  0.69480833,
        -0.23221018, -0.61498384],
       [ 0.73943397, -0.60205494, -0.92382264, ...,  0.69480833,
        -0.23221018, -0.61498384]])

In [25]:
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

log_clf = LogisticRegression()
svm_clf = SVC()
rfc_clf = RandomForestClassifier()
vot_clf = VotingClassifier(estimators=[('lr', log_clf), ('sc', svm_clf), ('rf', rfc_clf)],voting="hard")

vot_clf.fit(X_train, y_train)


VotingClassifier(estimators=[('lr', LogisticRegression()), ('sc', SVC()),
                             ('rf', RandomForestClassifier())])

In [41]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rfc_clf, svm_clf, vot_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.7906976744186046
RandomForestClassifier 0.7441860465116279
SVC 0.6976744186046512
VotingClassifier 0.7674418604651163
